In [3]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip -qq /content/drive/MyDrive/03-11.zip

replace __MACOSX/._03-11? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 03-11/Syn.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
!unzip -qq /content/drive/MyDrive/CSV-01-12.zip

In [4]:
!unzip -qq /content/drive/MyDrive/reference.zip

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
import sys

In [7]:
import os
task = 'train'
if task == 'train':
    train_flag = True
elif task == 'test':
    train_flag = False
    # model_path = sys.argv[4]
else:
    sys.exit('Invalid Operation')

os.mkdir('new_output_folder')
os.mkdir('save_outputs')

In [8]:
features_file = '/content/reference/features.txt'
run_no = '/content/new_output_folder'
print('running script run no: ' + run_no)


running script run no: /content/new_output_folder


In [9]:
max_val = 99999

with open(features_file) as f:
    features = [feature.strip() for feature in f]

if train_flag:
    with open('/content/reference/test_files.txt') as f:
        train_files = [filename.strip() for filename in f]

with open('/content/reference/train_files.txt') as f:
    test_files = [filename.strip() for filename in f]


    

In [13]:
test_dir = '/content/01-12/'
train_dir = '/content/03-11/'
output_dir ='/content/save_outputs/'

In [14]:
def read_file(filename, y_out):
    df = pd.read_csv(filename)
    df.columns = df.columns.str.strip()
    df = df[features] 
    NewLabel = []
    for i in df["Label"]:
        if i =="BENIGN":
            NewLabel.append(0)
        else:
            NewLabel.append(1)
    df["Label"]=NewLabel
    y = df['Label'].values
    y_out = y_out.extend(y)
    del df['Label']

    df = df.replace('Infinity', max_val)
    x = df.values
    scaler = QuantileTransformer(n_quantiles=1000, random_state=42)
    scaled_df = scaler.fit_transform(x)
    #scaled_df=(df-df.mean())/df.std()
    x = pd.DataFrame(scaled_df)
    return x
    

In [16]:
nClasses = 2
#### LOAD TRAIN DATA ######
if train_flag:

    new_x = pd.DataFrame()
    temp_y = []

    for f in train_files:
        print('Processing file ' + f + '\n')
        new_x = new_x.append(read_file(train_dir + f, temp_y))
        print('Processed file ' + f + ' , total samples is ' + str(len(temp_y)) + '\n')

    #new_y = np.asarray(temp_y)
    new_y = to_categorical(temp_y, num_classes=nClasses)

    xTrain, xVal, yTrain, yVal = train_test_split(new_x, new_y, test_size = 0.2, random_state = 42)
    #np.savetxt('train_idx', idx1)
    #np.savetxt('test_idx', idx2)

    print('train size: ', xTrain.shape)
    print('train labels: ', yTrain.shape)
    print('Valid size: ',  xVal.shape)
    print('Valid labels: ',  yVal.shape)


Processing file LDAP.csv



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processed file LDAP.csv , total samples is 2113234

Processing file MSSQL.csv



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processed file MSSQL.csv , total samples is 7889020

Processing file NetBIOS.csv



...

Processed file Syn.csv , total samples is 15857154

Processing file UDP.csv



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processed file UDP.csv , total samples is 19639360

Processing file UDPLag.csv



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Processed file UDPLag.csv , total samples is 20364525

train size:  (16291620, 19)
train labels:  (16291620, 2)
Valid size:  (4072905, 19)
Valid labels:  (4072905, 2)


In [ ]:
#### LOAD TEST DATA #######
new_x = pd.DataFrame()
temp_y = []

for f in test_files:
    print('Processing file ' + f + '\n')
    new_x = new_x.append(read_file(test_dir + f, temp_y))
    print('Processed file ' + f + ' , total samples is ' + str(len(temp_y)) + '\n')

xTest = np.asarray(new_x)
#yTest = np.asarray(temp_y)
yTest = to_categorical(temp_y, num_classes=nClasses)

#xTest, xTemp, yTest, yTemp = train_test_split(new_x, new_y, test_size = 0.01, random_state = 42)
#np.savetxt('train_idx', idx1)
#np.savetxt('test_idx', idx2)

print('test size: ',  xTest.shape)


In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=len(features)-1, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nClasses, activation='softmax'))
print(model.summary(90))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Model: "sequential"
__________________________________________________________________________________________
 Layer (type)                           Output Shape                        Param #       
 dense (Dense)                          (None, 64)                          1280          
                                                                                          
 dropout (Dropout)                      (None, 64)                          0             
                                                                                          
 dense_1 (Dense)                        (None, 64)                          4160          
                                                                                          
 dropout_1 (Dropout)                    (None, 64)                          0             
                                                                                          
 dense_2 (Dense)                        (None, 64)                    

In [ ]:
num_batch = 1000
num_epochs = 10
es = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=num_epochs, mode='auto', baseline=None, restore_best_weights=True, verbose=1)
if train_flag:

    model.fit(xTrain, yTrain, batch_size=num_batch, epochs = num_epochs, verbose=1, callbacks=[es], validation_data=[xVal, yVal])
    model.save(output_dir + run_no + '/model_weights')
else:
    model = load_model(model_path)
print(model.evaluate(xTest,yTest))


Epoch 1/10
16292/16292 [==============================] - 65s 4ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 2/10
16292/16292 [==============================] - 62s 4ms/step - loss: 0.0020 - accuracy: 0.9996
Epoch 3/10
16292/16292 [==============================] - 62s 4ms/step - loss: 0.0021 - accuracy: 0.9996
Epoch 4/10
16292/16292 [==============================] - 61s 4ms/step - loss: 0.0021 - accuracy: 0.9996
Epoch 5/10
16292/16292 [==============================] - 61s 4ms/step - loss: 0.0022 - accuracy: 0.9996
Epoch 6/10
16292/16292 [==============================] - 62s 4ms/step - loss: 0.0021 - accuracy: 0.9996
Epoch 7/10
16292/16292 [==============================] - 62s 4ms/step - loss: 0.0022 - accuracy: 0.9996
Epoch 8/10
16292/16292 [==============================] - 61s 4ms/step - loss: 0.0022 - accuracy: 0.9996
Epoch 9/10
16292/16292 [==============================] - 61s 4ms/step - loss: 0.0023 - accuracy: 0.9996
Epoch 10/10
16292/16292 [==============================

NameError: ignored

In [ ]:
prediction = np.argmax(model.predict(xTest), axis=1)
y_test = np.argmax(yTest, axis=1)
print('\n Accuracy: ')
print(accuracy_score(y_test, prediction))
print('\n F1 score: ')
print(f1_score(y_test, prediction))
print('\n Recall: ')
print(recall_score(y_test, prediction))
print('\n Precision: ')
print(precision_score(y_test, prediction))
print('\n confusion matrix: \n')
print(confusion_matrix(y_test, prediction))
np.savetxt(output_dir + run_no +'/predictions.txt', prediction)
np.savetxt(output_dir + run_no +'/ground_truth.txt', y_test)